<a href="https://colab.research.google.com/github/ravishankar75/group44_cv_assignment/blob/main/CV_assignment_1_group44.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Group 44

## Group Member Names:
1. SAKTHI R (2023aa05940)
2. ROBERTSEKAR R (2023aa05823)
3. RAVISHANKAR R (2023aa05171)
4. KRISHNAKUMAR C (2023aa05273)

Dataset Link: https://drive.google.com/file/d/18ivVD85YKQqPH0Qhe2Ou10hjuPl-vWxA/view?usp=sharing


Choose any 1 dataset of your choice to perform the assignment.

# 1. Import the required libraries -- Score: 0.5 Marks

In [14]:
import os
import zipfile
from google.colab import drive

import numpy as np
import pandas as pd

import cv2
from skimage.feature import local_binary_pattern #opencv2 does not expose LBP

# 2. Data Acquisition & Preparation -- Score: 1.5 Marks

For the problem identified by you, students have to find the data source themselves from any data source.

## 2.1 Data Acquisition -- Score: 0.5 Mark

Code for converting the above downloaded data into a form suitable for DL


In [6]:
##---------Type the code below this line------------------##

# Shared dataset link not working with gdown library due to permissions
# The images have been downloaded and uploaded to Google drive(My).

# The below steps necessary as on reconnect, google collab deletes the content folder

colab_folder = "/content"  # Default working directory in Google Colab
zip_file_name = "/content/drive/MyDrive/scene_classification.zip"


# Decompress the zip file, using with instead of try/finally
# check if data folder exists, if it exists then the dataset has been extracted

if not os.path.exists(os.path.join(colab_folder, "data")):

  print("Mounting Google Drive...")
  drive.mount('/content/drive') # Approve access for google desktop app

  print("Decompressing the zip file...")
  with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
      extract_path = os.path.join(colab_folder, "data")
      zip_ref.extractall(extract_path)

print( f"Decompression complete., Files extracted to {extract_path}")

# find the number of images for each category

for category in os.listdir(os.path.join(colab_folder, "data", "subset")):
  category_path = os.path.join(colab_folder, "data", "subset", category)
  if not os.path.isdir(category_path):
    continue
  num_images = len(os.listdir(category_path))
  print(f"Category: {category}, Number of Images: {num_images}")


Decompression complete., Files extracted to /content/data
Category: Forest, Number of Images: 2745
Category: Streets, Number of Images: 501
Category: Building, Number of Images: 501
Category: Sea, Number of Images: 501
Category: Mountains, Number of Images: 501
Category: Glacier, Number of Images: 501


In [16]:
# to delete after completion

# Define constants
NUM_IMAGES_PER_FOLDER = 5
DATASET = []

# Features to extract from images - Color, Shape and Texture

# Function to resize Image, convert all images to standard size
def resize_image(image):
    IMAGE_SIZE = (128, 128)  # Resize to 128x128
    resized_image = cv2.resize(image, IMAGE_SIZE)
    return resized_image

# Function to extract color details
# Forests are mostly green, Sea blue, Glaciers White etc
def extract_color_details(image):
   # OpenCv2 loads in BGR format, calculate histogram of intensity
   # Mask = None, All 3 channels, 8 Bins, Range of Intensity (0 to L-1) L=256
   hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
   hist = cv2.normalize(hist, hist).flatten()
   return hist

# Function to extract texture
# Texture refer to spatial arrangement of intensity or color patterns in an image
# Helps identify if there are irregular patterns (Mountain), uniform patterns (Sea, glacier), repeating patterns (Building, Street) etc
# Methods - Local Binary Factor, Gibor Filter, GLCM not incl- computational cost

def extract_texture_LBP(gray_image):
   # Input image has to be  grayscale
   # Compute the LBP of the image, using skimage.feature.localBinaryPattern
   # P=8, R=3.0
   lbp = local_binary_pattern(gray_image, 8, 3.0, method="uniform")

   return lbp

def extract_texture_gabor(gray_image, psi):
  # Input image has to be  grayscale
  # Apply Gabor kernel, from the resultant image pixels, calculate mean & stddev

   gabor_kernel = cv2.getGaborKernel(ksize=(5, 5), sigma=5.0, theta=0.0, lambd=10.0, gamma=0.5, psi=psi)

   filtered_image = cv2.filter2D(gray_image, cv2.CV_8UC3, gabor_kernel)

   mean = np.mean(filtered_image)
   std_dev = np.std(filtered_image)
   return (mean, std_dev)

# Function to extract shape features
# Determine edges, Buildings/Street should have more edges than Sea/Glacier
# Use measure = no of edge pixels / total pixels in image

def extract_edge_features(gray_image):

  # detect edges using Canny algo
  edge_image = cv2.Canny(gray_image, threshold1=50, threshold2=150)
  no_of_edge_pixel = np.Sum(edge_image > 0 ) # Any non black pixel

  return no_of_edge_pixel/gray_image.size

# Path to the base folder containing category subfolders
base_folder = os.path.join(colab_folder, "data", "subset")

# Iterate through category folders
for category in os.listdir(base_folder):
    category_path = os.path.join(base_folder, category)
    if not os.path.isdir(category_path):
        continue

    # Process up to NUM_IMAGES_PER_FOLDER images in this category
    image_count = 0
    for image_file in os.listdir(category_path):
        if image_count >= NUM_IMAGES_PER_FOLDER:
            break

        # Full path to image file
        image_path = os.path.join(category_path, image_file)
        if not os.path.isfile(image_path):
            continue

        # Process the image
        image = cv2.imread(image_path)

        if image is None:
            continue
        # Convert image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        dominant_colors = extract_color_details(image)
        texture_lbp = extract_texture_LBP(gray_image)
        texture_gabor_0 = extract_texture_gabor(gray_image, 0)
        texture_gabor_45 = extract_texture_gabor(gray_image, 45)
        texture_gabor_90 = extract_texture_gabor(gray_image, 90)

        edge_density = extract_edge_features(gray_image)


        # Append data to dataset
        DATASET.append({
            "filename": image_path,
            "label": category,
            "dominant_colors": dominant_colors.tolist(),
            "texture_lbp": texture_lbp.tolist(),
            "texture_gabor_0": texture_gabor_0,
            "texture_gabor_45": texture_gabor_45,
            "texture_gabor_90": texture_gabor_90,
            "edge_density": edge_density
        })
        image_count += 1

# Convert dataset to a DataFrame
df = pd.DataFrame(DATASET)

# Save dataset to CSV
df.to_csv("image_dataset.csv", index=False)

print("Dataset created and saved to 'image_dataset.csv'")




Dataset created and saved to 'image_dataset.csv'


## 2.2 Write your observations from the above.

1. Size of the dataset
2. Plot the distribution of the categories of the target / label.



## 2.2 Data Preparation -- Score: 1.0 Marks

Perform the data preprocessing that is required for the data that you have downloaded.


This stage depends on the dataset that is used.

## 3.1 Split the data into training set and testing set

In [ ]:
##---------Type the code below this line------------------##


## 3.2 Feature Engineering -- Score: 3.5 Marks

* Extract the features from the images and concatenate them to create a single for the every images.

* You can choose from the feature processing techniques taught in the class : Low-level Vision: Histogram and Histogram equalization, Gray-scale transformation, Image Smoothing, Connected components in images.
Mid-level Vision:  Edge Detection using Gradients, Sobel, Canny; Line detection using Hough transforms; Semantic information using RANSAC;Image region descriptor using SIFT; Use case: Pedestrian detection Using HoG and SIFT descriptors and SVM

* Create multiple sets of features and store it in seperate dataframes so that you can later use it for training and comparing the models.

* Normalize the DataFrame

* Note : If the feature size is getting too large such that it is not fitting into the RAM of Colab or your system then you can either use PCA or resize the image to smaller dimenssion for reducing the numer of features



In [ ]:
##---------Type the answer below this line------------------##

# 4. Model Building - Score: 2.0 Marks

## 4.1 Model Building - Score: 1.5 Marks
* Use any 1 classical machine learning algorithm such as : SVM , Xgboost etc. to train the model
* Train the model on different kinds of feature combination dataframe you created in 3.

In [ ]:
##---------Type the code below this line------------------##

## 4.2 Validation matrix - Score: 0.5 Marks

Print the model accuracy and F1 Score


In [ ]:
##---------Type the answer below this line------------------##

# 5. Model Inference & Evaluation - Score: 1 Mark

Plot any 5 random test images and their predicted and actual true labels using the model and feature set which gave you the best accuracy/F1 score.


In [ ]:
##---------Type the code below this line------------------##

Justify your choice/inution of feature selection based on the performance of model such that why a particualr set have features might have performed well.


In [ ]:
##---------Type the answers below this line------------------##

# 6. Documentation, Study presentation and Code Quality -- Score: 1.5 Marks

### NOTE


All Late Submissions will incur a <b>penalty of -2 marks </b>. So submit your assignments on time.

Good Luck